In [12]:
#importamos las librerias 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.datasets import load_digits

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA


from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

from sklearn.preprocessing import RobustScaler


from sklearn.metrics import root_mean_squared_error

 
from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split


from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [5]:
paht= './data/diamonds_train.csv'   #leemos el archivos csv que esta en la carpeta data
diamons_cleaning_2 = pd.read_csv(paht, sep= ",")

In [6]:
#Volumen: Calcular el volumen del diamante como una nueva característica:
diamons_cleaning_2['volume'] = diamons_cleaning_2['x'] * diamons_cleaning_2['y'] * diamons_cleaning_2['z']


In [8]:
#Volvemos las variables categoricas en numericas 
diamons_cleaning_2 = pd.get_dummies(diamons_cleaning_2, columns=["city","cut","color","clarity",], dtype=int)

In [9]:
diamons_cleaning_2

,price,carat,depth,table,x,y,z,volume,city_Amsterdam,city_Antwerp,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,4268,1.21,62.4,58.0,6.83,6.79,4.25,197.096725,0,0,...,0,1,0,0,0,0,0,1,0,0
1,505,0.32,63.0,57.0,4.35,4.38,2.75,52.395750,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2686,0.71,65.5,55.0,5.62,5.53,3.65,113.436890,0,0,...,0,0,0,0,0,0,1,0,0,0
3,738,0.41,63.8,56.0,4.68,4.72,3.00,66.268800,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4882,1.02,60.5,59.0,6.55,6.51,3.95,168.429975,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,10070,1.34,62.7,57.0,7.10,7.04,4.43,221.429120,0,1,...,0,0,0,0,0,0,1,0,0,0
40451,12615,2.02,57.1,60.0,8.31,8.25,4.73,324.276975,0,0,...,0,0,0,0,0,1,0,0,0,0
40452,5457,1.01,62.7,56.0,6.37,6.42,4.01,163.990554,0,0,...,0,0,0,0,1,0,0,0,0,0
40453,456,0.33,61.9,54.3,4.45,4.47,2.76,54.900540,0,0,...,0,1,0,0,0,0,1,0,0,0


# Hacemos el robust Scaler

In [10]:
#Seleccionar las columnas numéricas excepto price:
numerical_columns = ['carat', 'depth', 'table', 'x', 'y', 'z', 'volume']


In [16]:
#Aplicar el escalado a las columnas numéricas:
scaler = RobustScaler()
diamons_cleaning_2[numerical_columns] = scaler.fit_transform(diamons_cleaning_2[numerical_columns])


In [17]:
diamons_cleaning_2

,price,carat,depth,table,x,y,z,volume,city_Amsterdam,city_Antwerp,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,4268,0.796875,0.400000,0.333333,0.622951,0.593407,0.648889,0.777758,0,0,...,0,1,0,0,0,0,0,1,0,0
1,505,-0.593750,0.800000,0.000000,-0.732240,-0.730769,-0.684444,-0.587925,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2686,0.015625,2.466667,-0.666667,-0.038251,-0.098901,0.115556,-0.011821,0,0,...,0,0,0,0,0,0,1,0,0,0
3,738,-0.453125,1.333333,-0.333333,-0.551913,-0.543956,-0.462222,-0.456992,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4882,0.500000,-0.866667,0.666667,0.469945,0.439560,0.382222,0.507202,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,10070,1.000000,0.600000,0.000000,0.770492,0.730769,0.808889,1.007407,0,1,...,0,0,0,0,0,0,1,0,0,0
40451,12615,2.062500,-3.133333,1.000000,1.431694,1.395604,1.075556,1.978082,0,0,...,0,0,0,0,0,1,0,0,0,0
40452,5457,0.484375,0.600000,-0.333333,0.371585,0.390110,0.435556,0.465303,0,0,...,0,0,0,0,1,0,0,0,0,0
40453,456,-0.578125,0.066667,-0.900000,-0.677596,-0.681319,-0.675556,-0.564285,0,0,...,0,1,0,0,0,0,1,0,0,0


# Probamos el modelo 

In [20]:
#Separar las características (X) y el objetivo (y):
X= diamons_cleaning_2.drop('price', axis=1)
y = diamons_cleaning_2['price']

#Dividir el conjunto de datos en entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [24]:
#LightGBM:
model_5 = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=43)
model_5_cleaning_2.fit(X, y)
y_pred_5 = model_5_cleaning_2.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_5))
rmse

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002793 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1583
[LightGBM] [Info] Number of data points in the train set: 40455, number of used features: 40
[LightGBM] [Info] Start training from score 3928.444469


np.float64(480.57162368564644)

In [28]:
%%time
# Realizar validación cruzada y calcular el RMSE
scores = cross_val_score(model_5, 
                         X, 
                         y, 
                         scoring='neg_root_mean_squared_error', 
                         cv=5,
                         n_jobs=-1)

# Imprimir el tipo de modelo y los scores
print(type(model_5), '\n')
scores, 

# Calcular y mostrar el RMSE promedio
print(np.mean(-scores))


<class 'lightgbm.sklearn.LGBMRegressor'> 

552.0115775808797
CPU times: total: 125 ms
Wall time: 2.55 s


In [29]:
# Definir el modelo
model_5 = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=43)

# Realizar validación cruzada y calcular el RMSE
scores = cross_val_score(model_5, 
                         X, 
                         y, 
                         scoring='neg_root_mean_squared_error', 
                         cv=5,
                         n_jobs=-1)


In [30]:
# Imprimir el tipo de modelo y los scores
print(type(model_5), '\n')
print("Scores de validación cruzada (neg_root_mean_squared_error):", scores)

# Calcular y mostrar el RMSE promedio
rmse_promedio = np.mean(-scores)
print("RMSE promedio:", rmse_promedio)


<class 'lightgbm.sklearn.LGBMRegressor'> 

Scores de validación cruzada (neg_root_mean_squared_error): [-547.88408542 -531.02236477 -557.53145445 -589.79773199 -533.82225126]
RMSE promedio: 552.0115775808797
